In [1]:
import pandas as pd
import numpy as np
import json
from datetime import timedelta
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# 1. Wczytanie danych
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)
glucose_data['date'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])
glucose_data = glucose_data[['date', 'glucose']].dropna(subset=['date', 'glucose']).sort_values('date')

with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# 2. Przetwarzanie danych
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator and calculator.get('glucose') and calculator.get('units', {}).get('short'):
            date = pd.to_datetime(calculator['date'], errors='coerce').replace(tzinfo=None)
            insulin = calculator['units']['short']
            glucose_pre = calculator['glucose']
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            fats = sum(meal.get('fats', 0) for meal in record.get('meals', []))
            prot = sum(meal.get('prot', 0) for meal in record.get('meals', []))
            
            glucose_window = glucose_data[
                (glucose_data['date'] >= date + timedelta(hours=1)) & 
                (glucose_data['date'] <= date + timedelta(hours=2.5))
            ]
            if not glucose_window.empty:
                glucose_post = np.mean(glucose_window['glucose'].values)
                processed_data.append([date, glucose_pre, glucose_post, insulin, carbs, fats, prot])

df = pd.DataFrame(processed_data, columns=['date', 'glucose_pre', 'glucose_post', 'insulin', 'carbs', 'fats', 'prot'])

# 3. Przygotowanie cech
df['WW'] = df['carbs'] / 10
df['insulin_per_ww'] = np.where(df['WW'] > 0, df['insulin'] / df['WW'], 0)
df['hour'] = df['date'].dt.hour + df['date'].dt.minute / 60
df['part_of_day'] = df['date'].dt.hour.apply(lambda h: 0 if 6 <= h < 12 else 1 if 12 <= h < 18 else 2 if 18 <= h < 24 else 3)

# Obliczenie indywidualnego ICR
ok_glucose = df[(df['glucose_post'] >= 80) & (df['glucose_post'] <= 150)]
mean_icr = ok_glucose['insulin_per_ww'].mean() if not ok_glucose.empty else 1.0
print(f"Średni stosunek insuliny do WW z historii (ICR): {mean_icr:.2f}")

X = df[['glucose_pre', 'insulin', 'WW', 'fats', 'prot', 'insulin_per_ww', 'hour', 'part_of_day']]
y = df['glucose_post']

# Usunięcie inf i NaN
mask = ~(np.isinf(X).any(axis=1) | X.isna().any(axis=1) | np.isinf(y) | y.isna())
X = X[mask]
y = y[mask]

# Normalizacja
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Trenowanie modelu regresyjnego
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    reg_lambda=1.0,
    reg_alpha=0.1
)
model.fit(X_train, y_train)

# Walidacja krzyżowa
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
rmse_cv = np.sqrt(-cv_scores)
print(f"Średni RMSE (CV): {rmse_cv.mean():.2f} (+/- {rmse_cv.std() * 2:.2f})")

# 5. Klasyfikacja dawki i przewidywana dawka
def classify_dose_and_suggest(glucose_post, insulin, ww, glucose_pre, mean_icr):
    insulin_per_ww = insulin / ww if ww > 0 else 0
    target_glucose = 110
    sensitivity = 40

    # Klasyfikacja dawki
    if insulin_per_ww > 2 * mean_icr:
        feedback = 2
    elif insulin_per_ww < 0.5 * mean_icr and (glucose_post > 150 or (glucose_pre > 120 and glucose_post > 130)):
        feedback = -1
    elif insulin_per_ww < 0.5 * mean_icr and glucose_post > 180:
        feedback = -2
    elif glucose_post < 70:
        feedback = 2
    elif 70 <= glucose_post < 80:
        feedback = 1
    elif 80 <= glucose_post <= 150:
        feedback = 0
    elif 150 < glucose_post <= 180:
        feedback = -1
    elif 180 < glucose_post <= 240:
        feedback = -2
    else:
        feedback = -3

    # Przewidywana dawka
    glucose_diff = glucose_post - target_glucose
    correction = glucose_diff / sensitivity
    carb_insulin = ww * mean_icr
    pre_correction = max(0, (glucose_pre - 120) / sensitivity)
    suggested_insulin = max(0, insulin + correction + (carb_insulin - insulin) + pre_correction)

    return feedback, suggested_insulin

# 6. Testowanie
test_cases = [
    [150, 2, 5, 10, 5, 2/5 if 5 > 0 else 0, 9.5, 1],
    [100, 5, 3, 5, 2, 5/3 if 3 > 0 else 0, 12.0, 0],
    [200, 15, 6, 20, 10, 15/6 if 6 > 0 else 0, 18.5, 2],
    [90, 2, 2, 5, 2, 2/2 if 2 > 0 else 0, 22.0, 3],
    [180, 25, 5, 15, 8, 25/5 if 5 > 0 else 0, 8.0, 1],
    [140, 6, 4, 10, 5, 6/4 if 4 > 0 else 0, 14.0, 2],
]

test_df = pd.DataFrame(test_cases, columns=['glucose_pre', 'insulin', 'WW', 'fats', 'prot', 'insulin_per_ww', 'hour', 'part_of_day'])
test_scaled = scaler.transform(test_df)
glucose_predictions = model.predict(test_scaled)

feedback_desc = {
    -3: "Znacznie za mała dawka",
    -2: "Za mała dawka",
    -1: "Trochę za mała dawka",
    0: "OK",
    1: "Trochę za duża dawka",
    2: "Za duża dawka"
}

part_of_day_desc = {
    0: "Rano (6:00-11:59)",
    1: "Popołudnie (12:00-17:59)",
    2: "Wieczór (18:00-23:59)",
    3: "Noc (0:00-5:59)"
}

for i, (glucose_pred, test_case) in enumerate(zip(glucose_predictions, test_cases)):
    glucose_pre = test_case[0]
    insulin = test_case[1]
    ww = test_case[2]
    fats = test_case[3]
    prot = test_case[4]
    hour = test_case[6]
    part_of_day = test_case[7]
    feedback, suggested_insulin = classify_dose_and_suggest(glucose_pred, insulin, ww, glucose_pre, mean_icr)
    print(f"Test {i+1}:")
    print(f"  Wejście: glucose_pre={glucose_pre} mg/dL, insulin={insulin} jednostek, WW={ww}, fats={fats} g, prot={prot} g")
    print(f"  Czas: hour={hour:.1f}, part_of_day={part_of_day_desc[part_of_day]}")
    print(f"  Ocena dawki: {feedback_desc[feedback]} (insulin/WW: {test_case[5]:.2f})")
    print(f"  Sugerowana dawka: {suggested_insulin:.1f} jednostek")
    print(f"  Przewidywana glukoza: {glucose_pred:.1f} mg/dL")
    print()

# 7. Ocena modelu
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE na zbiorze testowym: {rmse:.2f} mg/dL")

Średni stosunek insuliny do WW z historii (ICR): 0.96
Średni RMSE (CV): 27.02 (+/- 7.14)
Test 1:
  Wejście: glucose_pre=150 mg/dL, insulin=2 jednostek, WW=5, fats=10 g, prot=5 g
  Czas: hour=9.5, part_of_day=Popołudnie (12:00-17:59)
  Ocena dawki: Trochę za mała dawka (insulin/WW: 0.40)
  Sugerowana dawka: 6.5 jednostek
  Przewidywana glukoza: 148.2 mg/dL

Test 2:
  Wejście: glucose_pre=100 mg/dL, insulin=5 jednostek, WW=3, fats=5 g, prot=2 g
  Czas: hour=12.0, part_of_day=Rano (6:00-11:59)
  Ocena dawki: OK (insulin/WW: 1.67)
  Sugerowana dawka: 3.8 jednostek
  Przewidywana glukoza: 145.5 mg/dL

Test 3:
  Wejście: glucose_pre=200 mg/dL, insulin=15 jednostek, WW=6, fats=20 g, prot=10 g
  Czas: hour=18.5, part_of_day=Wieczór (18:00-23:59)
  Ocena dawki: Za duża dawka (insulin/WW: 2.50)
  Sugerowana dawka: 9.0 jednostek
  Przewidywana glukoza: 160.1 mg/dL

Test 4:
  Wejście: glucose_pre=90 mg/dL, insulin=2 jednostek, WW=2, fats=5 g, prot=2 g
  Czas: hour=22.0, part_of_day=Noc (0:00-5:59)

In [2]:
import pandas as pd
import numpy as np
import json
from datetime import timedelta
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# 1. Wczytanie danych
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)
glucose_data['date'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])
glucose_data = glucose_data[['date', 'glucose']].dropna(subset=['date', 'glucose']).sort_values('date')

with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# 2. Przetwarzanie danych
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator and calculator.get('glucose') and calculator.get('units', {}).get('short'):
            date = pd.to_datetime(calculator['date'], errors='coerce').replace(tzinfo=None)
            insulin = calculator['units']['short']
            glucose_pre = calculator['glucose']
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            fats = sum(meal.get('fats', 0) for meal in record.get('meals', []))
            prot = sum(meal.get('prot', 0) for meal in record.get('meals', []))
            
            glucose_window = glucose_data[
                (glucose_data['date'] >= date + timedelta(hours=1)) & 
                (glucose_data['date'] <= date + timedelta(hours=2.5))
            ]
            if not glucose_window.empty:
                glucose_post = np.mean(glucose_window['glucose'].values)
                processed_data.append([date, glucose_pre, glucose_post, insulin, carbs, fats, prot])

df = pd.DataFrame(processed_data, columns=['date', 'glucose_pre', 'glucose_post', 'insulin', 'carbs', 'fats', 'prot'])

# 3. Przygotowanie cech
df['WW'] = df['carbs'] / 10
df['insulin_per_ww'] = np.where(df['WW'] > 0, df['insulin'] / df['WW'], 0)
df['hour'] = df['date'].dt.hour + df['date'].dt.minute / 60
df['part_of_day'] = df['date'].dt.hour.apply(lambda h: 0 if 6 <= h < 12 else 1 if 12 <= h < 18 else 2 if 18 <= h < 24 else 3)

# Obliczenie indywidualnego ICR
ok_glucose = df[(df['glucose_post'] >= 80) & (df['glucose_post'] <= 150)]
mean_icr = ok_glucose['insulin_per_ww'].mean() if not ok_glucose.empty else 1.0
print(f"Średni stosunek insuliny do WW z historii (ICR): {mean_icr:.2f}")

X = df[['glucose_pre', 'insulin', 'WW', 'fats', 'prot', 'insulin_per_ww', 'hour', 'part_of_day']]
y = df['glucose_post']

# Usunięcie inf i NaN
mask = ~(np.isinf(X).any(axis=1) | X.isna().any(axis=1) | np.isinf(y) | y.isna())
X = X[mask]
y = y[mask]

# Normalizacja
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Trenowanie modelu regresyjnego
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    reg_lambda=1.0,
    reg_alpha=0.1
)
model.fit(X_train, y_train)

# Walidacja krzyżowa
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
rmse_cv = np.sqrt(-cv_scores)
print(f"Średni RMSE (CV): {rmse_cv.mean():.2f} (+/- {rmse_cv.std() * 2:.2f})")

# 5. Klasyfikacja dawki i przewidywana dawka
def classify_dose_and_suggest(glucose_post, insulin, ww, glucose_pre, mean_icr):
    insulin_per_ww = insulin / ww if ww > 0 else 0
    target_glucose = 110
    sensitivity = 40

    # Klasyfikacja dawki
    if insulin_per_ww > 2 * mean_icr:
        feedback = 2
    elif insulin_per_ww < 0.5 * mean_icr and (glucose_post > 150 or (glucose_pre > 120 and glucose_post > 130)):
        feedback = -1
    elif insulin_per_ww < 0.5 * mean_icr and glucose_post > 180:
        feedback = -2
    elif glucose_post < 70:
        feedback = 2
    elif 70 <= glucose_post < 80:
        feedback = 1
    elif 80 <= glucose_post <= 150:
        feedback = 0
    elif 150 < glucose_post <= 180:
        feedback = -1
    elif 180 < glucose_post <= 240:
        feedback = -2
    else:
        feedback = -3

    # Przewidywana dawka
    glucose_diff = glucose_post - target_glucose
    correction = glucose_diff / sensitivity  # Dodatnia jeśli za wysoka, ujemna jeśli za niska
    carb_insulin = ww * mean_icr  # Bazowa dawka na WW
    pre_correction = max(0, (glucose_pre - 120) / sensitivity)  # Korekta za wysoką glukozę przed
    suggested_insulin = max(0, carb_insulin + pre_correction + correction)  # Nowa logika

    return feedback, suggested_insulin

# 6. Testowanie
test_cases = [
    [150, 2, 5, 10, 5, 2/5 if 5 > 0 else 0, 9.5, 1],
    [100, 5, 3, 5, 2, 5/3 if 3 > 0 else 0, 12.0, 0],
    [200, 15, 6, 20, 10, 15/6 if 6 > 0 else 0, 18.5, 2],
    [90, 2, 2, 5, 2, 2/2 if 2 > 0 else 0, 22.0, 3],
    [180, 25, 5, 15, 8, 25/5 if 5 > 0 else 0, 8.0, 1],
    [140, 6, 4, 10, 5, 6/4 if 4 > 0 else 0, 14.0, 2],
]

test_df = pd.DataFrame(test_cases, columns=['glucose_pre', 'insulin', 'WW', 'fats', 'prot', 'insulin_per_ww', 'hour', 'part_of_day'])
test_scaled = scaler.transform(test_df)
glucose_predictions = model.predict(test_scaled)

feedback_desc = {
    -3: "Znacznie za mała dawka",
    -2: "Za mała dawka",
    -1: "Trochę za mała dawka",
    0: "OK",
    1: "Trochę za duża dawka",
    2: "Za duża dawka"
}

part_of_day_desc = {
    0: "Rano (6:00-11:59)",
    1: "Popołudnie (12:00-17:59)",
    2: "Wieczór (18:00-23:59)",
    3: "Noc (0:00-5:59)"
}

for i, (glucose_pred, test_case) in enumerate(zip(glucose_predictions, test_cases)):
    glucose_pre = test_case[0]
    insulin = test_case[1]
    ww = test_case[2]
    fats = test_case[3]
    prot = test_case[4]
    hour = test_case[6]
    part_of_day = test_case[7]
    feedback, suggested_insulin = classify_dose_and_suggest(glucose_pred, insulin, ww, glucose_pre, mean_icr)
    print(f"Test {i+1}:")
    print(f"  Wejście: glucose_pre={glucose_pre} mg/dL, insulin={insulin} jednostek, WW={ww}, fats={fats} g, prot={prot} g")
    print(f"  Czas: hour={hour:.1f}, part_of_day={part_of_day_desc[part_of_day]}")
    print(f"  Ocena dawki: {feedback_desc[feedback]} (insulin/WW: {test_case[5]:.2f})")
    print(f"  Sugerowana dawka: {suggested_insulin:.1f} jednostek")
    print(f"  Przewidywana glukoza: {glucose_pred:.1f} mg/dL")
    print()

# 7. Ocena modelu
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE na zbiorze testowym: {rmse:.2f} mg/dL")

Średni stosunek insuliny do WW z historii (ICR): 0.96
Średni RMSE (CV): 27.02 (+/- 7.14)
Test 1:
  Wejście: glucose_pre=150 mg/dL, insulin=2 jednostek, WW=5, fats=10 g, prot=5 g
  Czas: hour=9.5, part_of_day=Popołudnie (12:00-17:59)
  Ocena dawki: Trochę za mała dawka (insulin/WW: 0.40)
  Sugerowana dawka: 6.5 jednostek
  Przewidywana glukoza: 148.2 mg/dL

Test 2:
  Wejście: glucose_pre=100 mg/dL, insulin=5 jednostek, WW=3, fats=5 g, prot=2 g
  Czas: hour=12.0, part_of_day=Rano (6:00-11:59)
  Ocena dawki: OK (insulin/WW: 1.67)
  Sugerowana dawka: 3.8 jednostek
  Przewidywana glukoza: 145.5 mg/dL

Test 3:
  Wejście: glucose_pre=200 mg/dL, insulin=15 jednostek, WW=6, fats=20 g, prot=10 g
  Czas: hour=18.5, part_of_day=Wieczór (18:00-23:59)
  Ocena dawki: Za duża dawka (insulin/WW: 2.50)
  Sugerowana dawka: 9.0 jednostek
  Przewidywana glukoza: 160.1 mg/dL

Test 4:
  Wejście: glucose_pre=90 mg/dL, insulin=2 jednostek, WW=2, fats=5 g, prot=2 g
  Czas: hour=22.0, part_of_day=Noc (0:00-5:59)